In [ ]:
# Train YOLOv10 to Recognize 5 People Using Folder Names as Labels (From Google Drive)

In [ ]:
# 1. Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 2. Copy data from Google Drive to local Colab workspace
!cp -r /content/drive/MyDrive/train_yolov10 /content/face_data_raw

In [ ]:
# 3. Install Ultralytics
!pip install ultralytics -q

In [ ]:
# 4. Import Libraries
import os
from glob import glob
from PIL import Image
from pathlib import Path
from sklearn.model_selection import train_test_split

In [ ]:
# 5. Set Paths
data_raw = Path('/content/face_data_raw')  # your input folder
output_dir = Path('/content/datasets/face_id_5people')
img_train_dir = output_dir / 'images/train'
img_val_dir = output_dir / 'images/val'
lbl_train_dir = output_dir / 'labels/train'
lbl_val_dir = output_dir / 'labels/val'

In [ ]:
for d in [img_train_dir, img_val_dir, lbl_train_dir, lbl_val_dir]:
    d.mkdir(parents=True, exist_ok=True)

In [ ]:
# 6. Collect image paths and labels
all_data = []
label_map = {}

In [ ]:
for idx, person_dir in enumerate(sorted(data_raw.iterdir())):
    if person_dir.is_dir():
        label_map[person_dir.name] = idx
        for img_path in glob(str(person_dir / '*.jpg')):
            all_data.append((img_path, idx))

In [ ]:
# 7. Train-val split
train_data, val_data = train_test_split(all_data, test_size=0.2, stratify=[d[1] for d in all_data], random_state=42)

In [ ]:
# 8. Save images and labels in YOLO format
def process_data(data_list, img_dir, lbl_dir):
    for img_path, label in data_list:
        img = Image.open(img_path)
        w, h = img.size
        yolo_label = f"{label} 0.5 0.5 1.0 1.0\n"
        img_name = Path(img_path).name
        img.save(img_dir / img_name)
        with open(lbl_dir / img_name.replace('.jpg', '.txt'), 'w') as f:
            f.write(yolo_label)

In [ ]:
process_data(train_data, img_train_dir, lbl_train_dir)
process_data(val_data, img_val_dir, lbl_val_dir)

In [ ]:
# 9. Create .yaml config file
yaml_path = output_dir / 'face5.yaml'
names_list = list(label_map.keys())
yaml_lines = [
    f"path: {output_dir}",
    "train: images/train",
    "val: images/val",
    f"names: {names_list}"
]
with open(yaml_path, 'w') as f:
    f.write('\n'.join(yaml_lines))

In [ ]:
# 10. Train YOLOv10n (lightweight)
from ultralytics import YOLO
model = YOLO('yolov10n.pt')
model.train(data=str(yaml_path), epochs=50, imgsz=640, batch=16)

In [ ]:
# 11. Copy the model to main directory
!cp runs/detect/train/weights/best.pt /content/best_face_yolov10.pt

In [ ]:
# 12. (Optional) Predict test image
# model.predict(source='path_to_test_image.jpg', conf=0.25, save=True)